# Benchmarking Our Algorithm against others

In [3]:
import sys
sys.path.append('../')
from panav.environment.env import MultiTunnelEnv
from panav.hybrid import HybridGraph
from panav.util import flowtime,makespan
from panav.PBS.PBS import PBS
from panav.TrafficAwarePlanning import TAHP
from panav.sequential import sequential_planning
from panav.PIBT import PIBT_plan
from panav.SAMP.solvers import Tube_Planning



from time import time
import pickle as pkl

%load_ext autoreload
%autoreload 2

In [1]:

bloating_r = 0.5
vmax = 1.0

# n_agents = list(range(10,101,10))
# limits = [(-20,20),(-80,80)] 
# n_tunnel = 20


n_agents = list(range(2,21,2))
limits = [(-10,10),(-10,10)] 
n_tunnel = 4

# n_agents = [6]

TIMEOUT = 100 # timeout

# Algorithms
algs = {  "PIBT": lambda hg: PIBT_plan(hg,vmax,bloating_r,TIMEOUT),
        "S2M2": lambda hg: PBS(hg.env,vmax=vmax,bloating_r=bloating_r,low_level_planner="Simple_MILP_Planning",TIMEOUT=TIMEOUT)[0],
        "PBS_Tube": lambda hg: PBS(hg.env,vmax=vmax,bloating_r=bloating_r,low_level_planner="Tube_Planning",TIMEOUT=TIMEOUT)[0],
        "TAHP": lambda hg: TAHP(hg,vmax=vmax,bloating_r=bloating_r,TIMEOUT=TIMEOUT),
        "Seq_Tube": lambda hg: sequential_planning(Tube_Planning,hg.env,vmax,bloating_r,TIMEOUT=TIMEOUT),      
        # "Seq_Temp_Obs": lambda hg: sequential_planning(Simple_MILP_Planning,hg.env,vmax,bloating_r,TIMEOUT=TIMEOUT)
        }

timeouted = {alg:False for alg in algs}

results = []

env_name = "MultiTunnel"


In [3]:
# Load checkpoint if necessary

with open("./data/{}_misc.pkl".format(env_name),'rb') as fp:
    ck = pkl.load(fp)
    cur_N = ck['cur_N']



with open("./data/{}.pkl".format(env_name),'rb') as fp:
    results = pkl.load(fp)

In [4]:
cur_N = 0
# Experiments
for i in range(len(n_agents)):
    
    N = n_agents[i]
    if N<cur_N:
        continue


    env = MultiTunnelEnv(n_tunnel = n_tunnel, tunnel_width = bloating_r*2.5,limits=limits, N_agent = N)

    HG = HybridGraph(env,bloating_r)

    # Save checkpoint data
    with open("./data/{}_misc.pkl".format(env_name),'wb') as fp:
        pkl.dump({"TIMEOUT":TIMEOUT,"HG":HG,'cur_N':N},fp)
        
    to_remove = []
    for e in HG.edges:
        if HG.edges[e]['type']=='soft':
            # HG.edges[e]['weight'] = 0
            # pass
            u,v = e
            if HG.nodes[u]['type']== HG.nodes[v]['type']=='tunnel' and \
                                    HG.nodes[u]['open_space_id'] == HG.nodes[v]['open_space_id']:
                to_remove.append(e) # Remove the soft edges connecting two tunnel endpoints for this particular environment
    for e in to_remove:
        HG.remove_edge(*e)

    for name,alg in algs.items():

        if timeouted[name]:
            continue
        
        print("Alg:", name, "N agent:",N)

        result = {}
        
        t0 = time()
        plan = alg(HG)
        t = time()-t0
        
        result['plan'] = plan
        result['time'] = t

        result['N_agent'] = N
        result['alg_name'] = name

        result["TIMEOUT"]=TIMEOUT
        result["HG"]=HG

        results.append(result)

        
        if t>TIMEOUT:
            timeouted[name] = True
            print("Time out for algorithm", name, "N agent = ", N)


        
        

    # Save checkpoint data
    with open("./data/{}.pkl".format(env_name),'wb') as fp:
        pkl.dump(results,fp)


Alg: PIBT N agent: 2
Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-15


/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/shapely/measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


Alg: S2M2 N agent: 2
PBS Time Elapsed: 0.22731709480285645
PBS Time Elapsed: 0.31136488914489746
Alg: PBS_Tube N agent: 2
PBS Time Elapsed: 0.1524829864501953
PBS Time Elapsed: 0.21820688247680664
Alg: TAHP N agent: 2
Planning for 0/2
Planning for 1/2
Alg: Seq_Tube N agent: 2
Planning for agent 0/2
Planning for agent 1/2
Alg: PIBT N agent: 4
Alg: S2M2 N agent: 4
PBS Time Elapsed: 0.2677450180053711
PBS Time Elapsed: 0.3523252010345459
PBS Time Elapsed: 0.4654829502105713
PBS Time Elapsed: 0.5906820297241211
PBS Time Elapsed: 0.7150661945343018
PBS Time Elapsed: 0.9737980365753174
PBS Time Elapsed: 1.095707893371582
PBS Time Elapsed: 1.3672130107879639
PBS Time Elapsed: 1.450711965560913
PBS Time Elapsed: 1.6017191410064697
PBS Time Elapsed: 1.842320203781128
PBS Time Elapsed: 1.9258182048797607
PBS Time Elapsed: 2.080796957015991
PBS Time Elapsed: 2.3180510997772217
Alg: PBS_Tube N agent: 4
PBS Time Elapsed: 0.2732810974121094
PBS Time Elapsed: 0.3450031280517578
PBS Time Elapsed: 0.41

/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/cvxpy/problems/problem.py:1385: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Alg: PBS_Tube N agent: 10
PBS Time Elapsed: 0.5857348442077637
PBS Time Elapsed: 0.656641960144043
PBS Time Elapsed: 0.7316930294036865
PBS Time Elapsed: 0.7797996997833252
PBS Time Elapsed: 0.8576807975769043
PBS Time Elapsed: 1.0037829875946045
PBS Time Elapsed: 1.1439030170440674
PBS Time Elapsed: 1.2624597549438477
PBS Time Elapsed: 1.5081028938293457
PBS Time Elapsed: 1.6213970184326172
PBS Time Elapsed: 1.7415838241577148
PBS Time Elapsed: 1.9263648986816406
PBS Time Elapsed: 2.025930881500244
PBS Time Elapsed: 2.180840015411377
PBS Time Elapsed: 2.224565029144287
PBS Time Elapsed: 2.359869956970215
PBS Time Elapsed: 2.4891648292541504
PBS Time Elapsed: 2.615196943283081
PBS Time Elapsed: 2.7572379112243652
PBS Time Elapsed: 2.8047139644622803
PBS Time Elapsed: 2.947927713394165
PBS Time Elapsed: 3.3542709350585938
PBS Time Elapsed: 3.5559067726135254
PBS Time Elapsed: 3.601104974746704
PBS Time Elapsed: 3.7420568466186523
PBS Time Elapsed: 3.8692739009857178
PBS Time Elapsed: 3.

/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/cvxpy/problems/problem.py:1391: UserWarning: 
    The problem is either infeasible or unbounded, but the solver
    cannot tell which. Disable any solver-specific presolve methods
    and re-solve to determine the precise problem status.

    For GUROBI and CPLEX you can automatically perform this re-solve
    with the keyword argument prob.solve(reoptimize=True, ...).
    
  warnings.warn(INF_OR_UNB_MESSAGE)


Alg: PBS_Tube N agent: 12
PBS Time Elapsed: 0.6380999088287354
PBS Time Elapsed: 0.7069740295410156
PBS Time Elapsed: 0.7731361389160156
PBS Time Elapsed: 0.8192791938781738
PBS Time Elapsed: 0.9254651069641113
PBS Time Elapsed: 1.0529699325561523
PBS Time Elapsed: 1.1611509323120117
PBS Time Elapsed: 1.276386022567749
PBS Time Elapsed: 1.3806209564208984
PBS Time Elapsed: 1.4896960258483887
PBS Time Elapsed: 1.6662802696228027
PBS Time Elapsed: 1.91131591796875
PBS Time Elapsed: 2.039170026779175
PBS Time Elapsed: 2.1338911056518555
PBS Time Elapsed: 2.30806303024292
PBS Time Elapsed: 2.351517915725708
PBS Time Elapsed: 2.482835054397583
PBS Time Elapsed: 2.725304126739502
PBS Time Elapsed: 2.8840219974517822
PBS Time Elapsed: 2.961121082305908
PBS Time Elapsed: 3.1035890579223633
PBS Time Elapsed: 3.2456822395324707
PBS Time Elapsed: 3.4102320671081543
PBS Time Elapsed: 3.455307960510254
PBS Time Elapsed: 3.6164159774780273
PBS Time Elapsed: 3.7699859142303467
PBS Time Elapsed: 3.943